# Monte Carlo Insurance Risk Simulation

This notebook demonstrates a Monte Carlo simulation for insurance risk analysis, which can be used to model the financial outcomes of an insurance portfolio.

The simulation generates random claim events and amounts based on probability distributions to estimate expected profits, potential losses, and key risk metrics.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set a seed for reproducibility
np.random.seed(42)

## Simulation Parameters

Let's set up the parameters for our insurance portfolio simulation:

In [ ]:
# Parameters for our insurance scenario
num_simulations = 10000
num_policies = 1000

# Insurance parameters
annual_premium = 1200  # Annual premium per policy ($)
max_claim_payout = 100000  # Maximum claim payout ($)
avg_claim_amount = 20000  # Average claim amount ($)
claim_std_dev = 15000  # Standard deviation of claim amounts
claim_probability = 0.05  # Probability of a claim being filed

## Running the Monte Carlo Simulation

We'll run multiple simulations to model the distribution of possible outcomes:

In [ ]:
# Monte Carlo simulation
results = []

for sim in range(num_simulations):
    # Generate random number of claims (using binomial distribution)
    num_claims = np.random.binomial(n=num_policies, p=claim_probability)
    
    # Generate random claim amounts (using lognormal distribution to skew towards smaller claims)
    mu = np.log(avg_claim_amount) - 0.5 * np.log(1 + (claim_std_dev/avg_claim_amount)**2)
    sigma = np.sqrt(np.log(1 + (claim_std_dev/avg_claim_amount)**2))
    claim_amounts = np.random.lognormal(mean=mu, sigma=sigma, size=num_claims)
    
    # Cap claim amounts at the maximum payout
    claim_amounts = np.minimum(claim_amounts, max_claim_payout)
    
    # Calculate total revenue and payouts
    total_revenue = annual_premium * num_policies
    total_claims = np.sum(claim_amounts)
    profit = total_revenue - total_claims
    
    # Store results
    results.append({
        'simulation': sim,
        'num_claims': num_claims,
        'total_revenue': total_revenue,
        'total_claims': total_claims,
        'profit': profit
    })

# Convert results to DataFrame
df_results = pd.DataFrame(results)

## Analyzing the Results

Let's calculate key risk metrics:

In [ ]:
# Calculate key metrics
expected_profit = df_results['profit'].mean()
profit_std = df_results['profit'].std()
var_95 = np.percentile(df_results['profit'], 5)  # 95% Value at Risk
cvar_95 = df_results[df_results['profit'] <= var_95]['profit'].mean()  # Conditional VaR

# Probability of loss
prob_loss = (df_results['profit'] < 0).mean() * 100

# Display results
print(f"Monte Carlo Simulation Results ({num_simulations} simulations)")
print(f"Number of policies: {num_policies}")
print(f"Annual premium per policy: ${annual_premium}")
print(f"Claim probability: {claim_probability:.1%}")
print(f"Average claim amount: ${avg_claim_amount}")
print(f"---")
print(f"Expected annual profit: ${expected_profit:,.2f}")
print(f"Profit standard deviation: ${profit_std:,.2f}")
print(f"Probability of loss: {prob_loss:.2f}%")
print(f"95% Value at Risk: ${-var_95:,.2f}")
print(f"95% Conditional VaR: ${-cvar_95:,.2f}")

## Visualizing the Results

Now let's create visualizations to better understand the risk profile:

In [ ]:
# Visualizations
plt.figure(figsize=(12, 10))

# Histogram of profits
plt.subplot(2, 2, 1)
sns.histplot(df_results['profit'], kde=True)
plt.axvline(x=0, color='red', linestyle='--')
plt.axvline(x=expected_profit, color='green', linestyle='-')
plt.title('Distribution of Profits')
plt.xlabel('Profit ($)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Histogram of number of claims
plt.subplot(2, 2, 2)
sns.histplot(df_results['num_claims'], kde=True, discrete=True)
plt.axvline(x=df_results['num_claims'].mean(), color='green', linestyle='-')
plt.title('Distribution of Number of Claims')
plt.xlabel('Number of Claims')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Scatter plot of claims vs. profit
plt.subplot(2, 2, 3)
plt.scatter(df_results['total_claims'], df_results['profit'], alpha=0.3)
plt.title('Relationship Between Claims and Profit')
plt.xlabel('Total Claims ($)')
plt.ylabel('Profit ($)')
plt.grid(True, alpha=0.3)

# CDF of profits
plt.subplot(2, 2, 4)
sorted_profits = np.sort(df_results['profit'])
cumulative_prob = np.arange(len(sorted_profits)) / float(len(sorted_profits))
plt.plot(sorted_profits, cumulative_prob)
plt.axvline(x=0, color='red', linestyle='--')
plt.axvline(x=var_95, color='orange', linestyle='--', label='95% VaR')
plt.title('Cumulative Distribution of Profits')
plt.xlabel('Profit ($)')
plt.ylabel('Probability')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

## Sensitivity Analysis

Let's examine how changing the premium affects expected profit:

In [ ]:
# Sensitivity analysis - Premium vs. Profit
premium_range = np.linspace(800, 1600, 20)
expected_profits = []

for premium in premium_range:
    profits = []
    for _ in range(1000):  # Smaller number of simulations for speed
        num_claims = np.random.binomial(n=num_policies, p=claim_probability)
        claim_amounts = np.random.lognormal(mean=mu, sigma=sigma, size=num_claims)
        claim_amounts = np.minimum(claim_amounts, max_claim_payout)
        total_revenue = premium * num_policies
        total_claims = np.sum(claim_amounts)
        profit = total_revenue - total_claims
        profits.append(profit)
    expected_profits.append(np.mean(profits))

plt.figure(figsize=(10, 6))
plt.plot(premium_range, expected_profits)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Annual Premium ($)')
plt.ylabel('Expected Profit ($)')
plt.title('Sensitivity Analysis: Premium vs. Expected Profit')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Conclusion

This Monte Carlo simulation helps us understand:  
  
1. The expected annual profit from our insurance portfolio  
2. The range of potential outcomes and their probabilities  
3. The risk of significant losses (Value at Risk)  
4. How sensitive our profitability is to premium pricing

Based on this analysis, we can make more informed decisions about premium pricing, risk management strategies, and capital reserves needed to maintain solvency.